# Brazilian E-Commerce Olist 데이터 분석
#### - 패스트캠퍼스 데이터 분석 부트캠프 17기
#### - 파이썬 프로젝트 3조 : 김유정, 김민경, 한서희, 안지연, 김유림

## 데이터 전처리
#### 2016년 전체, 2018년 9월 데이터 제거 / 각 데이터셋 병합 / USD 환산

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [ ]:
customer = pd.read_csv('olist_customers_dataset.csv')
geolocation = pd.read_csv('olist_geolocation_dataset.csv')
item = pd.read_csv('olist_order_items_dataset.csv')
payment = pd.read_csv('olist_order_payments_dataset.csv')
order = pd.read_csv('olist_orders_dataset.csv', parse_dates=['order_purchase_timestamp'])
product = pd.read_csv('olist_products_dataset.csv')
translation = pd.read_csv('product_category_name_translation_new.csv')
review = pd.read_csv('olist_order_reviews_dataset.csv')
segmentation = pd.read_csv('new_product_category_segmentation.csv')

customer_order = pd.merge(customer, order, on='customer_id', how='inner')
customer_order_item = pd.merge(customer_order, item, on='order_id', how='inner')

exchange_rate = 0.2404
customer_order_item['price_usd'] = (customer_order_item['price'] * exchange_rate).astype(int)

columns_to_keep = ['customer_id', 'customer_unique_id', 'order_id','customer_state', 'order_purchase_timestamp', 'order_item_id', 'price_usd', 'product_id']
customer_order_item = customer_order_item[columns_to_keep]

customer_order_item.loc[(customer_order_item['order_purchase_timestamp'].dt.year == 2016) |
((customer_order_item['order_purchase_timestamp'].dt.year == 2018) &
(customer_order_item['order_purchase_timestamp'].dt.month == 9)), 'order_purchase_timestamp'] = pd.NaT

merged_cleaned = customer_order_item.dropna(subset=['order_purchase_timestamp'])

product = product.dropna(subset=['product_category_name'])

trans_seg = pd.merge(translation, segmentation, on='product_category_name_english', how='left')
trans_seg_product = pd.merge(product, trans_seg, on='product_category_name', how='left')

columns_to_keep = ['product_id', 'product_category_name_english', 'segmentation']
trans_seg_product = trans_seg_product[columns_to_keep]

merged_cleaned = pd.merge(customer_order_item, trans_seg_product, on='product_id', how='inner')

merged_cleaned.to_csv('merged_cleaned.csv', index=False)

## 현황분석 (1)

### 매출 추이 (2017.01~2018.08)

In [ ]:
# 'order_purchase_timestamp' 컬럼을 datetime 타입으로 변환
merged_cleaned['order_purchase_timestamp'] = pd.to_datetime(merged_cleaned['order_purchase_timestamp'])

# 연도 및 월을 추출하여 새로운 컬럼 추가
merged_cleaned['year_month'] = merged_cleaned['order_purchase_timestamp'].dt.to_period('M')

# 월별 매출 합계 및 판매량 계산
monthly_sales_volume = merged_cleaned.groupby('year_month').agg(
    total_sales=('price_usd', 'sum'),  # 총 매출
    total_sales_volume=('order_item_id', 'count')  # 총 판매량 (order_item_id로 계산)
).reset_index()

# 그래프 생성
fig, ax1 = plt.subplots(figsize=(14, 8))

# 매출 (Bar Chart)
ax1.bar(monthly_sales_volume['year_month'].astype(str), 
        monthly_sales_volume['total_sales'], 
        color='#87CEEB',  # 바그래프 색상 변경
        label='Total Sales')
ax1.set_ylabel('Total Sales', fontsize=14)
ax1.tick_params(axis='x', rotation=45, labelsize=14)
ax1.tick_params(axis='y')

# 판매량 (Line Chart)
ax2 = ax1.twinx()
ax2.plot(monthly_sales_volume['year_month'].astype(str), 
         monthly_sales_volume['total_sales_volume'], 
         color='#8B0000',  # 선그래프 색상 변경
         marker='o', 
         label='Sales Count')
ax2.set_ylabel('Sales Count', fontsize=14)
ax2.tick_params(axis='y')

# 매출에 대한 추세선 추가
x_values = np.arange(len(monthly_sales_volume))  # 월별 인덱스를 x값으로 사용
y_values = monthly_sales_volume['total_sales']  # 매출 값
z = np.polyfit(x_values, y_values, 1)  # 1차 선형 회귀선
p = np.poly1d(z)  # 회귀 함수 생성

# 추세선 추가
ax1.plot(monthly_sales_volume['year_month'].astype(str), p(x_values), 
         color='#08306b', linestyle='-', label='Sales Trend', linewidth=2)

# 범례 병합 및 순서 정렬
handles1, labels1 = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
handles = handles1 + handles2
labels = labels1 + labels2

desired_order = ['Total Sales', 'Sales Count', 'Sales Trend']
sorted_handles_labels = sorted(zip(handles, labels), key=lambda x: desired_order.index(x[1]))
sorted_handles, sorted_labels = zip(*sorted_handles_labels)
ax1.legend(sorted_handles, sorted_labels, loc='upper left', fontsize=14)

# 여백 자동 조정
plt.tight_layout()

# 이미지(.png) 저장
plt.savefig('Monthly_Sales_Count.png', dpi=300, bbox_inches='tight', transparent=True)

# 월별 매출 및 판매량 그래프 출력
plt.show()

In [ ]:
# 연도 및 분기를 추출하여 새로운 컬럼 추가
merged_cleaned['quarter'] = merged_cleaned['order_purchase_timestamp'].dt.to_period('Q')

# 각 주문 항목의 개수(판매량)를 세서 분기별로 합계 계산
quarterly_sales = merged_cleaned.groupby('quarter').agg(
    total_sales=('price_usd', 'sum'),
    total_sales_volume=('order_item_id', 'count')  # 각 주문 항목 개수를 세서 판매량 계산
).reset_index()

# 2017Q2 실적 추출
target_sales_2017Q2 = quarterly_sales[quarterly_sales['quarter'] == '2017Q2']['total_sales'].values[0]

# 그래프 생성
fig, ax1 = plt.subplots(figsize=(14, 8))

# 매출 (Bar Chart)
bars = ax1.bar(quarterly_sales['quarter'].astype(str), 
               quarterly_sales['total_sales'], 
               color='#87CEEB', 
               label='Total Sales')

ax1.set_ylabel('Total Sales', fontsize=14)
ax1.tick_params(axis='x', labelsize=14)  # x축 라벨의 기본 글씨 크기 설정
plt.xticks(ticks=range(len(quarterly_sales['quarter'])),  # x축의 틱 위치 설정
           labels=quarterly_sales['quarter'].astype(str),  # x축 라벨 설정
           fontsize=14)  # 글씨 크기 설정

# 판매량 (Line Chart)
ax2 = ax1.twinx()  # 두 번째 y축
ax2.plot(quarterly_sales['quarter'].astype(str), 
         quarterly_sales['total_sales_volume'], 
         color='#8B0000', 
         marker='o', 
         label='Sales Count', linewidth=2)
ax2.set_ylabel('Sales Count', fontsize=14)

# 2018Q3 위에 빗금박스 표시 (2017Q2 실적을 기준으로 목표 도달량 표시)
q3_index = quarterly_sales[quarterly_sales['quarter'] == '2018Q3'].index[0]
target_sales_2017Q2 = quarterly_sales[quarterly_sales['quarter'] == '2017Q2']['total_sales'].values[0]

# x, y 위치를 맞추기 위한 박스 설정
ax1.axvspan(q3_index - 0.4, q3_index + 0.4, 
             facecolor='none', edgecolor='#87CEEB', hatch='//', linewidth=2, 
             ymin=0, ymax=quarterly_sales['total_sales'].max() / ax1.get_ylim()[1])  # y값을 매출 범위에 맞춰서 설정

# 목표 도달량 "-38.77%" 텍스트 추가
ax1.text(q3_index, 
         quarterly_sales['total_sales'][q3_index] * 1.30,  # 텍스트를 박스 위에 약간 띄움
         "-38.77%", 
         color='#8B0000',  # 텍스트 색상 라인 색상과 동일
         fontsize=22, 
         fontweight='bold',  # 글자를 두껍게 설정
         ha='center', 
         va='bottom')

# 범례 병합 및 순서 정렬
handles1, labels1 = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
handles = handles1 + handles2
labels = labels1 + labels2

desired_order = ['Total Sales', 'Total Sales Volume']
# 정렬 로직 수정: desired_order에 없는 레이블은 뒤에 배치
sorted_handles_labels = sorted(zip(handles, labels), key=lambda x: desired_order.index(x[1]) if x[1] in desired_order else float('inf'))
sorted_handles, sorted_labels = zip(*sorted_handles_labels)
ax1.legend(sorted_handles, sorted_labels, loc='upper left', fontsize=14)

# 한글 폰트 설정
plt.rc('font', family='Malgun Gothic')  # 맑은 고딕

# 유니코드 minus 기호 깨짐 방지
plt.rc('axes', unicode_minus=False)

# 2018Q3 그래프 앞에 박스 추가
ax1.text(q3_index +0.005, 
         quarterly_sales['total_sales'][q3_index] * 0.49,  # 텍스트 위치를 그래프 막대 위로
         '9월 데이터\n제외된 수치', 
         fontsize=12, 
         fontweight='bold', 
         color='#08306b', 
        bbox=dict(facecolor='none', edgecolor='#08306b', boxstyle='round,pad=0.7', linewidth=1.5),  # 박스 스타일 설정
         ha='center',  # 수평 중앙 정렬
         va='center')  # 수직 중앙 정렬

# 여백 자동 조정
plt.tight_layout()

# 이미지(.png) 저장
plt.savefig('Quarterly_Sales_Count.png', dpi=300, bbox_inches='tight', transparent=True)

# 분기별 매출 및 판매량 그래프 출력
plt.show()

### 카테고리 별 매출

#### - 제품 카테고리 별 매출 비율 (%)

In [ ]:
#(민경)복사본 생성
merged_category = merged_cleaned.copy()

#(민경)필요한 컬럼 생성
merged_category['order_purchase_timestamp'] = pd.to_datetime(merged_category['order_purchase_timestamp'])
merged_category['year']= merged_category['order_purchase_timestamp'].dt.year
merged_category['month']= merged_category['order_purchase_timestamp'].dt.month
merged_category['dayofweek']= merged_category['order_purchase_timestamp'].dt.dayofweek
merged_category['hour']= merged_category['order_purchase_timestamp'].dt.hour
merged_category['YM']= merged_category['order_purchase_timestamp'].dt.strftime('%Y-%m')

#파이차트
sum_price_seg = merged_category.groupby('segmentation')['price_usd'].sum().sort_values()
total_price = sum_price_seg.sum()

# segmentation별 비율 계산
sum_price_seg_percentage = (sum_price_seg / total_price) * 100  # 백분율로 변환

# 결과 출력
sum_price_seg_percentage = sum_price_seg_percentage.reset_index(name='percentage')

# 'Etc'로 묶인 값들 다시 그룹화
sum_price_seg_percentage_grouped = (
    sum_price_seg_percentage.groupby('segmentation', as_index=False)['percentage']
    .sum()
)

# 상위 5개 세그먼트 정의
top_segments = ['Electronics&Computers', 'Home&Garden', 'Health&Beauty', 'Sports&Leisure', 'Gifts&Occasions']

# 상위 5개 세그먼트와 나머지 그룹으로 데이터 변환
sum_price_seg_percentage_grouped['group'] = sum_price_seg_percentage_grouped['segmentation'].apply(
    lambda x: x if x in top_segments else 'Others'
)

# 그룹별 합계 계산
grouped_data = sum_price_seg_percentage_grouped.groupby('group', as_index=False)['percentage'].sum()

# 값이 작은 것부터 정렬
grouped_data = grouped_data.sort_values(by='percentage')

# 파란색 계열 색상 팔레트 설정 (상위 5개)
blue_palette = sns.color_palette("Blues", len(top_segments))

# 색상 설정: 상위 5개는 파란색 계열, 나머지는 회색
colors = blue_palette + ['whitesmoke']

# 원형 그래프 그리기
plt.figure(figsize=(8, 8))
plt.pie(
    grouped_data['percentage'],  # 값
    labels=grouped_data['group'],  # 라벨
    autopct='%1.1f%%',  # 퍼센트 표시 형식
    startangle=215,  # 12시 시작
    colors=colors  # 색상 설정
)

# plt.savefig('/content/drive/MyDrive/project/그래프/12_piechart_category_Sales.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

#### - 제품 카테고리 별 매출 & 판매량

In [ ]:
# 카데고리별 제품별 매출과 매출건수
total_sum = merged_cleaned.groupby('segmentation')['price_usd'].sum().sort_values(ascending = False)
total_count = merged_cleaned.groupby('segmentation').size().reset_index(name='count')  # count 열로 결과를 저장
seg_sum_count = pd.merge(total_sum, total_count, on='segmentation')
seg_sum_count['unit price'] = seg_sum_count['price_usd'] / seg_sum_count['count']
seg_sum_count

# 카테고리별 매출과 매출건수 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import pandas as pd

total_sum = merged_cleaned.groupby('segmentation')['price_usd'].sum().sort_values(ascending = False) # 판매합계
total_count = merged_cleaned.groupby('segmentation').size().reset_index(name='count')  # 주문건수
seg_sum_count = pd.merge(total_sum, total_count, on='segmentation') # 판매합계 + 주문건수

# 시각화
fig, ax1 = plt.subplots(figsize=(14, 8))

# 막대 그래프 (총 판매 금액)
sns.barplot(
    y='segmentation', 
    x='price_usd', 
    data=seg_sum_count, 
    palette=sns.color_palette('Blues', n_colors=len(seg_sum_count))[::-1], # 팔레트 반전
    # label='Total Sales',
    hue='segmentation',  # hue 설정
    dodge=False,         # 겹치지 않도록 설정
    legend=False,        # 범례 제거
    ax=ax1
)

ax1.set_ylabel('Product Category')
ax1.set_xlabel('Total Sales')
ax1.tick_params(axis='y', labelsize=15)
ax1.xaxis.set_major_locator(ticker.MultipleLocator(100000))  # 주요 눈금선 100,000 단위

# 보조 x축 생성
ax2 = ax1.twiny()  # 새로운 x축 추가

# 선 그래프 (주문 건수) - 보조 x축에 그리기
ax2.plot(
    seg_sum_count['count'], 
    seg_sum_count['segmentation'], 
    color='#8B0000', 
    marker='o', 
    linestyle='-', 
    linewidth=2, 
    label='Order Count'
)

# 보조 x축 설정
ax2.set_xlabel('Order Count')
ax2.tick_params(axis='x')
ax2.xaxis.set_major_locator(ticker.MultipleLocator(5000))  # 주요 눈금선 1,000 단위


# y축 눈금 정렬
ax1.yaxis.set_tick_params(labelsize=14)

# 저장 및 출력
plt.tight_layout()
plt.savefig('Categories_with_Bar_and_Line_Chart.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

#### - 제품 카테고리 별 매출 & 구매량 Top 5

In [ ]:
# 상위 5개 카테고리 이름 추출
top5_seg_price_sum = seg_sum_count.sort_values(by='price_usd', ascending=False).head(5)
top5_seg_price_sum

# 상위 5개 카테고리 이름 추출
top5_categories = top5_seg_price_sum['segmentation']

# category_seg 데이터에서 상위 5개 카테고리 필터링 및 집계
top5_seg_sum = (
    merged_cleaned[merged_cleaned['segmentation'].isin(top5_categories)]
    .groupby('segmentation', as_index=False)
    .agg({'price_usd': 'sum', 'order_id': 'count'})
    .nlargest(5, 'price_usd')
)

# 단위 가격 계산
top5_seg_sum['unit_price'] = (top5_seg_sum['price_usd'] / top5_seg_sum['order_id'])
top5_seg_sum

# 카테고리별 주문합계 / 주문건수 Top 5
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

# 시각화
fig, ax1 = plt.subplots(figsize=(10, 6))

# 막대 그래프 (총 판매 금액)
sns.barplot(
    x='segmentation', 
    y='price_usd', 
    data=top5_seg_sum, 
    palette='Blues_r', 
    hue='segmentation',  # hue 설정
    dodge=False,         # 겹치지 않도록 설정
    legend=False,        # 범례 제거
    ax=ax1
)
ax1.set_ylabel('Total Sales')
ax1.set_xlabel('Product Category')

# y축 눈금선 간격 조정
ax1.yaxis.set_major_locator(ticker.MultipleLocator(100000))  # 주요 눈금선 100,000 단위

# 선 그래프 (주문 건수)
ax2 = ax1.twinx()  # y축 공유
ax2.plot(
    top5_seg_sum['segmentation'], 
    top5_seg_sum['order_id'], 
    color='#8B0000', 
    marker='o', 
    linestyle='-', 
    linewidth=2, 
    label='Order Count'
)
ax2.set_ylabel('Order Count')
ax2.yaxis.set_major_locator(ticker.MultipleLocator(5000))

# 저장 및 출력
plt.tight_layout()
plt.savefig('Top_5_Categories.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
#전체 매출
YM_price_category = merged_category.groupby(['YM','segmentation'])['price_usd'].sum().reset_index()

# YM과 segmentation 기준으로 그룹화하고 등장 횟수 계산
YM_count_category = (
    merged_category
    .groupby(['YM', 'segmentation'])
    .size()
    .reset_index(name='count')  # count 열로 결과를 저장
)

# 특정 색상 지정
custom_colors = {
    "Electronics&Computers": "#4C72B0",  # 파랑
    "Furniture": "tab:purple",             # 주황
    "Gifts&Occasions": "#55A868",       # 초록
    "Health&Beauty": "tab:red",         # 빨강
    "Home&Garden": "darkorange",           # 보라
    "Sports&Leisure": "#937860"         # 갈색
}

col = {'Gifts&Occasions',
 'Electronics&Computers',
 'Furniture',
 'Health&Beauty',
 'Home&Garden',
 'Sports&Leisure'}

# 특정 카테고리 값만 필터링
filtered_data = YM_price_category[YM_price_category['segmentation'].isin(col)]

# 그래프 크기 설정
plt.figure(figsize=(12, 6))

# 선 그래프 생성
sns.lineplot(
    data=filtered_data,
    x='YM',
    y='price_usd',
    hue='segmentation',  # 카테고리별 색상 분리
    marker='o',  # 데이터 포인트 표시
    palette= custom_colors  # 색상 팔레트 설정
)

# 그래프 제목 및 라벨 설정
plt.ylabel("Total Sales", fontsize=12)
plt.xticks(rotation=45)
plt.xlabel("")  # x축 라벨 제거

# 범례 위치 설정
plt.legend(title="Product Category", loc='upper left', bbox_to_anchor=(1, 1))

# 레이아웃 조정 및 출력
plt.tight_layout()
# plt.savefig('/content/drive/MyDrive/project/그래프/15_YM_price_all.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
#Electronics&Computers, Home&Garden 매출
YM_price_category = merged_category.groupby(['YM','segmentation'])['price_usd'].sum().reset_index()

# YM과 segmentation 기준으로 그룹화하고 등장 횟수 계산
YM_count_category = (
    merged_category
    .groupby(['YM', 'segmentation'])
    .size()
    .reset_index(name='count')  # count 열로 결과를 저장
)

# 특정 색상 지정
custom_colors = {
    "Electronics&Computers": "#4C72B0",  # 파랑
    "Furniture": "gray",             # 주황
    "Gifts&Occasions": "gray",       # 초록
    "Health&Beauty": "gray",         # 빨강
    "Home&Garden": "darkorange",           # 보라
    "Sports&Leisure": "gray"         # 갈색
}


col = {'Gifts&Occasions',
 'Electronics&Computers',
 'Furniture',
 'Health&Beauty',
 'Home&Garden',
 'Sports&Leisure'}

# 특정 카테고리 값만 필터링
filtered_data = YM_price_category[YM_price_category['segmentation'].isin(col)]

# 그래프 크기 설정
plt.figure(figsize=(12, 6))

# 선 그래프 생성 (선 두껍게 및 마커 크게 설정)
for seg in filtered_data['segmentation'].unique():
    seg_data = filtered_data[filtered_data['segmentation'] == seg]
    if seg in ["Electronics&Computers", "Home&Garden"]:
        # Electronics&Computers와 Home&Garden의 선을 굵게, 마커 크게
        sns.lineplot(
            data=seg_data,
            x='YM',
            y='price_usd',
            label=seg,
            color=custom_colors[seg],  # 지정한 색상 사용
            linewidth=3.5,  # 굵은 선
        )
        plt.scatter(
            x=seg_data['YM'],
            y=seg_data['price_usd'],
            color=custom_colors[seg],
            s=40,  # 큰 마커 크기
            zorder=5,  # 다른 선 위에 표시
        )
    else:
        # 나머지 segmentation의 선은 기본 설정
        sns.lineplot(
            data=seg_data,
            x='YM',
            y='price_usd',
            label=seg,
            color=custom_colors[seg],  # 지정한 색상 사용
            linewidth=1.5,  # 기본 선 굵기
        )

# 그래프 제목 및 라벨 설정
plt.ylabel("Total Sales", fontsize=12)
plt.xticks(rotation=45)
plt.xlabel("")  # x축 라벨 제거

# 범례 위치 설정
plt.legend(title="Product Category", loc='upper left', bbox_to_anchor=(1, 1))

# 레이아웃 조정 및 출력
plt.tight_layout()
# plt.savefig('/content/drive/MyDrive/project/그래프/16_YM_price_Elec&Home.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
#Health&Beauty 매출
YM_price_category = merged_category.groupby(['YM','segmentation'])['price_usd'].sum().reset_index()

# YM과 product_category_name_english 기준으로 그룹화하고 등장 횟수 계산
YM_count_category = (
    merged_category
    .groupby(['YM', 'segmentation'])
    .size()
    .reset_index(name='count')  # count 열로 결과를 저장
)

# 특정 색상 지정 (segmentation의 순서에 맞게 지정)
custom_colors = {
    "Electronics&Computers": "grey",  # 파랑
    "Furniture": "grey",             # 주황
    "Gifts&Occasions": "grey",       # 초록
    "Health&Beauty": "tab:red",         # 빨강
    "Home&Garden": "grey",           # 보라
    "Sports&Leisure": "grey"         # 갈색
}

col = {'Gifts&Occasions',
 'Electronics&Computers',
 'Furniture',
 'Health&Beauty',
 'Home&Garden',
 'Sports&Leisure'}

# 특정 카테고리 값만 필터링
filtered_data = YM_price_category[YM_price_category['segmentation'].isin(col)]

# 그래프 크기 설정
plt.figure(figsize=(12, 6))


# 선 그래프 생성 (Health&Beauty 선 두껍게 및 마커 크게 설정)
for seg in filtered_data['segmentation'].unique():
    seg_data = filtered_data[filtered_data['segmentation'] == seg]
    line = sns.lineplot(
        data=seg_data,
        x='YM',
        y='price_usd',
        label=seg,
        color=custom_colors[seg],  # 지정한 색상 사용
        linewidth=3.5 if seg == "Health&Beauty" else 1.5,  # Health&Beauty만 두껍게
    )
    # Health&Beauty의 마커 크기 크게 설정
    if seg == "Health&Beauty":
        plt.scatter(
            x=seg_data['YM'],
            y=seg_data['price_usd'],
            color=custom_colors[seg],
            s=40,  # 마커 크기 (기본 크기보다 큼)
            zorder=5,  # 다른 선 위에 표시
        )

# 그래프 제목 및 라벨 설정

plt.ylabel("Total Sales", fontsize=12)
plt.xticks(rotation=45)
plt.xlabel("")  # x축 라벨 제거

# 범례 위치 설정
plt.legend(title="Product Category", loc='upper left', bbox_to_anchor=(1, 1))

# 레이아웃 조정 및 출력
plt.tight_layout()
# plt.savefig('/content/drive/MyDrive/project/그래프/17_YM_price_health.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

#### - 제품 카테고리 별 매출 & 구매량

In [ ]:
#전체 판매량
YM_price_category = merged_category.groupby(['YM','segmentation'])['price_usd'].sum().reset_index()

# YM과 segmentation 기준으로 그룹화하고 등장 횟수 계산
YM_count_category = (
    merged_category
    .groupby(['YM', 'segmentation'])
    .size()
    .reset_index(name='count')  # count 열로 결과를 저장
)

# 특정 색상 지정 (segmentation의 순서에 맞게 지정)
custom_colors = {
    "Electronics&Computers": "#4C72B0",  # 파랑
    "Furniture": "tab:purple",             # 주황
    "Gifts&Occasions": "#55A868",       # 초록
    "Health&Beauty": "tab:red",         # 빨강
    "Home&Garden": "darkorange",           # 보라
    "Sports&Leisure": "#937860"         # 갈색
}

col = {'Gifts&Occasions',
 'Electronics&Computers',
 'Furniture',
 'Health&Beauty',
 'Home&Garden',
 'Sports&Leisure'}

# 특정 카테고리 값만 필터링
filtered_data = YM_count_category[YM_count_category['segmentation'].isin(col)]

# 그래프 크기 설정
plt.figure(figsize=(12, 6))

# 선 그래프 생성
sns.lineplot(
    data=filtered_data,
    x='YM',
    y='count',
    hue='segmentation',  # 카테고리별 색상 분리
    marker='o',  # 데이터 포인트 표시
    palette= custom_colors  # 색상 팔레트 설정
)

# 그래프 제목 및 라벨 설정
plt.ylabel("Sales Count", fontsize=12)
plt.xticks(rotation=45)
plt.xlabel("")  # x축 라벨 제거

# 범례 위치 설정
plt.legend(title="Product Category", loc='upper left', bbox_to_anchor=(1, 1))

# 레이아웃 조정 및 출력
plt.tight_layout()
# plt.savefig('/content/drive/MyDrive/project/그래프/15_YM_count_all.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
#Electronics&Computers, Home&Garden 판매량
YM_price_category = merged_category.groupby(['YM','segmentation'])['price_usd'].sum().reset_index()

# YM과 segmentation 기준으로 그룹화하고 등장 횟수 계산
YM_count_category = (
    merged_category
    .groupby(['YM', 'segmentation'])
    .size()
    .reset_index(name='count')  # count 열로 결과를 저장
)

# 특정 색상 지정 (segmentation의 순서에 맞게 지정)
custom_colors = {
    "Electronics&Computers": "#4C72B0",  # 파랑
    "Furniture": "gray",             # 주황
    "Gifts&Occasions": "gray",       # 초록
    "Health&Beauty": "gray",         # 빨강
    "Home&Garden": "darkorange",           # 보라
    "Sports&Leisure": "gray"         # 갈색
}

col = {'Gifts&Occasions',
 'Electronics&Computers',
 'Furniture',
 'Health&Beauty',
 'Home&Garden',
 'Sports&Leisure'}

# 특정 카테고리 값만 필터링
filtered_data = YM_count_category[YM_count_category['segmentation'].isin(col)]

# 그래프 크기 설정
plt.figure(figsize=(12, 6))

# 선 그래프 생성 (선 두껍게 및 마커 크게 설정)
for seg in filtered_data['segmentation'].unique():
    seg_data = filtered_data[filtered_data['segmentation'] == seg]
    if seg in ["Electronics&Computers", "Home&Garden"]:
        # Electronics&Computers와 Home&Garden의 선을 굵게, 마커 크게
        sns.lineplot(
            data=seg_data,
            x='YM',
            y='count',
            label=seg,
            color=custom_colors[seg],  # 지정한 색상 사용
            linewidth=3.5,  # 굵은 선
        )
        plt.scatter(
            x=seg_data['YM'],
            y=seg_data['count'],
            color=custom_colors[seg],
            s=40,  # 큰 마커 크기
            zorder=5,  # 다른 선 위에 표시
        )
    else:
        # 나머지 segmentation의 선은 기본 설정
        sns.lineplot(
            data=seg_data,
            x='YM',
            y='count',
            label=seg,
            color=custom_colors[seg],  # 지정한 색상 사용
            linewidth=1.5,  # 기본 선 굵기
        )

# 그래프 제목 및 라벨 설정
plt.ylabel("Sales count", fontsize=12)
plt.xticks(rotation=45)
plt.xlabel("")  # x축 라벨 제거

# 범례 위치 설정
plt.legend(title="Product Category", loc='upper left', bbox_to_anchor=(1, 1))

# 레이아웃 조정 및 출력
plt.tight_layout()
# plt.savefig('/content/drive/MyDrive/project/그래프/16_YM_count_Elec&Home.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
#Health&Beauty 판매량
YM_price_category = merged_category.groupby(['YM','segmentation'])['price_usd'].sum().reset_index()

# YM과 product_category_name_english 기준으로 그룹화하고 등장 횟수 계산
YM_count_category = (
    merged_category
    .groupby(['YM', 'segmentation'])
    .size()
    .reset_index(name='count')  # count 열로 결과를 저장
)

# 특정 색상 지정 (segmentation의 순서에 맞게 지정)
custom_colors = {
    "Electronics&Computers": "grey",  # 파랑
    "Furniture": "grey",             # 주황
    "Gifts&Occasions": "grey",       # 초록
    "Health&Beauty": "tab:red",         # 빨강
    "Home&Garden": "grey",           # 보라
    "Sports&Leisure": "grey"         # 갈색
}

col = {'Gifts&Occasions',
 'Electronics&Computers',
 'Furniture',
 'Health&Beauty',
 'Home&Garden',
 'Sports&Leisure'}

# 특정 카테고리 값만 필터링
filtered_data = YM_count_category[YM_count_category['segmentation'].isin(col)]

# 그래프 크기 설정
plt.figure(figsize=(12, 6))

# 선 그래프 생성 (Health&Beauty 선 두껍게 및 마커 크게 설정)
for seg in filtered_data['segmentation'].unique():
    seg_data = filtered_data[filtered_data['segmentation'] == seg]
    line = sns.lineplot(
        data=seg_data,
        x='YM',
        y='count',
        label=seg,
        color=custom_colors[seg],  # 지정한 색상 사용
        linewidth=3.5 if seg == "Health&Beauty" else 1.5,  # Health&Beauty만 두껍게
    )
    # Health&Beauty의 마커 크기 크게 설정
    if seg == "Health&Beauty":
        plt.scatter(
            x=seg_data['YM'],
            y=seg_data['count'],
            color=custom_colors[seg],
            s=40,  # 마커 크기 (기본 크기보다 큼)
            zorder=5,  # 다른 선 위에 표시
        )

# 그래프 제목 및 라벨 설정
plt.ylabel("Sales Count", fontsize=12)
plt.xticks(rotation=45)
plt.xlabel("")  # x축 라벨 제거

# 범례 위치 설정
plt.legend(title="Product Category", loc='upper left', bbox_to_anchor=(1, 1))

# 레이아웃 조정 및 출력
plt.tight_layout()
# plt.savefig('/content/drive/MyDrive/project/그래프/17_YM_count_health.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

#### - 제품 카테고리 별 매출 평균 대비 매출 증감율

In [ ]:
#전체
YM_price_category = merged_category.groupby(['YM','segmentation'])['price_usd'].sum().reset_index()

df = YM_price_category.groupby('segmentation')['price_usd'].mean().reset_index()
# df의 price 컬럼 이름 변경
df.rename(columns={'price_usd': 'price_mean'}, inplace=True)
# segmentation 열 기준으로 YM_price_category와 df 병합
YM_price_category = YM_price_category.merge(df, on='segmentation', how='left')

# price_mean 대비 price의 증감율 계산
YM_price_category['price_mean_pct_change'] = (
    (YM_price_category['price_usd'] - YM_price_category['price_mean']) / YM_price_category['price_mean'] * 100
)

unique_segments = YM_price_category['segmentation'].unique()

for segment in unique_segments:
    # 각 세그먼트 데이터 필터링
    segment_data = YM_price_category[YM_price_category['segmentation'] == segment]

    # x값과 y값 추출
    x_values = segment_data['YM']
    y_values = segment_data['price_mean_pct_change']

    # x값을 숫자형으로 변환 (추세선 계산을 위해)
    x_numeric = np.arange(len(x_values))

    # 추세선 계수 계산
    trend_coeffs = np.polyfit(x_numeric, y_values, 1)
    trend_line = np.polyval(trend_coeffs, x_numeric)

    # 그래프 생성
    plt.figure(figsize=(12, 6))
    sns.lineplot(
        data=segment_data,
        x='YM',
        y='price_mean_pct_change',
        marker='o',
        color='#6baed6',
        label='Actual Data'
    )

    # 추세선 추가
    plt.plot(x_values, trend_line, color='#8B0000', linestyle='--', linewidth=1.5, label='Trend Line')

    # y=0 회색 실선 추가
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.8, alpha=0.7)

    # 그래프 제목 및 라벨 설정
    plt.title(f"Price Mean % Change for Segmentation: {segment}", fontsize=14)
    plt.xlabel("")
    plt.ylabel("Price Mean % Change (%)", fontsize=12)
    plt.xticks(rotation=45)

    # 범례 추가
    plt.legend()

    # 레이아웃 조정 및 출력
    plt.tight_layout()
    plt.show()

In [ ]:
# Food&Drink 평균대비 매출 증감율
segment = "Food&Drink"
segment_data = YM_price_category[YM_price_category['segmentation'] == segment]

# x값과 y값 추출
x_values = segment_data['YM']
y_values = segment_data['price_mean_pct_change']

# x값을 숫자형으로 변환 (추세선 계산을 위해)
x_numeric = np.arange(len(x_values))

# 추세선 계수 계산
trend_coeffs = np.polyfit(x_numeric, y_values, 1)
trend_line = np.polyval(trend_coeffs, x_numeric)

# 그래프 생성
plt.figure(figsize=(12, 6))
sns.lineplot(
    data=segment_data,
    x='YM',
    y='price_mean_pct_change',
    marker='o',
    color='#6baed6',
    label='Sales Mean % Change'
)

# 추세선 추가
plt.plot(x_values, trend_line, color='#8B0000', linestyle='--', linewidth=1.5, label='Trend Line')

# y=0 회색 실선 추가
plt.axhline(0, color='gray', linestyle='-', linewidth=0.8, alpha=0.7)

# 그래프 제목 및 라벨 설정
plt.xlabel("")
plt.ylabel("Sales Mean % Change (%)", fontsize=12)
plt.xticks(rotation=45)

# 범례 추가
plt.legend()

# 레이아웃 조정 및 출력
plt.tight_layout()
# plt.savefig('/content/drive/MyDrive/project/그래프/18_Sales_mean_Food&Drink.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

#### - 제품 카테고리 별 구매건당 매출

In [ ]:
# 특정 세그먼트에 대한 색상 설정 및 기본값
highlight_segment = "Gifts&Occasions"
default_color = "gray"
highlight_color = "#55A868"

# YM과 segmentation 열을 기준으로 데이터프레임 병합
merged_data = YM_price_category.merge(
    YM_count_category[['YM', 'segmentation', 'count']],  # 필요한 열만 선택
    on=['YM', 'segmentation'],  # 병합 기준 열
    how='left'  # 'left'로 설정하여 YM_price_category를 기준으로 병합
)

merged_data['sales_per_purchase'] = merged_data['price_usd'] / merged_data['count']

# 그래프 크기 설정
plt.figure(figsize=(12, 6))

# 각 세그먼트에 대해 선 그래프 생성
for seg in merged_data['segmentation'].unique():
    seg_data = merged_data[merged_data['segmentation'] == seg]
    sns.lineplot(
        data=seg_data,
        x='YM',
        y='sales_per_purchase',
        label=seg,
        color=highlight_color if seg == highlight_segment else default_color,
        linewidth=3.5 if seg == highlight_segment else 1.5,
        marker='o' if seg == highlight_segment else None  # 마커 설정
    )
    # 마커 크기 설정 (Gifts&Occasions만 크게)
    if seg == highlight_segment:
        plt.scatter(
            x=seg_data['YM'],
            y=seg_data['sales_per_purchase'],
            color=highlight_color,
            s=40,  # 마커 크기 (기본 크기보다 큼)
            zorder=5,  # 다른 선 위에 표시
        )

# 그래프 제목 및 라벨 설정
plt.xlabel("", fontsize=12)
plt.ylabel("Sales per purchase", fontsize=12)
plt.xticks(rotation=45)

# 범례 위치 설정
plt.legend(title="Product Category", loc='upper left', bbox_to_anchor=(1, 1))

# 레이아웃 조정 및 출력
plt.tight_layout()
# plt.savefig('/content/drive/MyDrive/project/그래프/19_Sales_per_perchase.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

### 리뷰 평점과의 상관관계

In [ ]:
total_sum = merged_cleaned.groupby('segmentation')['price_usd'].sum()
total_count = merged_cleaned.groupby('segmentation').size().reset_index(name='count')  # count 열로 결과를 저장

# 병합
merged_totals = pd.merge(total_sum, total_count, on='segmentation', how='inner')
merged_totals['unit_price'] = merged_totals['price_usd'] / merged_totals['count']
merged_totals

# 필요한 열만 선택
review_selected = review[['order_id', 'review_score']]
# review_selected에서 중복된 order_id 제거 (첫 번째 값만 유지)
review_selected_unique = review_selected.drop_duplicates(subset='order_id', keep='first')
# 병합 수행 (중복 제거 후)
merged_with_reviews = merged_cleaned.merge(review_selected_unique, on='order_id', how='left')  # left join으로 병합

# 결과 확인
merged_with_reviews
review_score = merged_with_reviews.groupby('segmentation')['review_score'].mean().reset_index().sort_values('review_score',ascending = False)
review_score
merged_totals = merged_totals.merge(review_score, on = 'segmentation')
merged_totals

# 상관관계 계산
correlation_matrix = merged_totals[["unit_price", "review_score", "count"]].corr()
# 히트맵 시각화
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="Blues_r", fmt=".2f", annot_kws={"size": 14}, cbar=True)
plt.savefig('avg_unit_price_heatmap.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

## 현황분석 (2) - RFM 모델

### Recency / Frequency / Monetary

#### - Recency

In [ ]:
# 분석 기준일을 설정 (2018년 9월 1일)
analysis_date = datetime(2018, 9, 1)

# 고객 고유 ID별로 가장 최근 주문 날짜를 기준으로 Recency(최종 구매일로부터 분석 기준일까지의 일수)를 계산
recency_df = merged_cleaned.groupby('customer_unique_id').agg(
    recency = ('order_purchase_timestamp', lambda x: (analysis_date - x.max()).days if pd.notna(x.max()) else None)
).reset_index()

# Recency 값을 기준으로 내림차순 정렬 (최근에 구매한 고객이 위에 오도록)
recency_df_f = recency_df.sort_values(by='recency', ascending=False).dropna(subset=['recency'])

# Recency 값에 따라 고객 수를 세어서 각 Recency 값에 해당하는 고객 수를 출력
recency_df_f.groupby('recency').size().sort_values(ascending=False)

# recency 값을 구간으로 나누기
bins = [0, 30, 90, 180, 365, float('inf')]  # 한 달, 세 달, 여섯 달, 1년 이상의 구간
labels = ['1month', '3month', '6month', '1year', '1year+']

# recency 값을 구간으로 분할
recency_df_f['recency'] = pd.cut(recency_df_f['recency'], bins=bins, labels=labels, right=True)

# 각 구간별 고객 수 계산
recency_grouped = recency_df_f.groupby('recency').size()

# 각 구간별 고객 수의 백분율 계산
recency_percentage = (recency_grouped / recency_grouped.sum()) * 100

# 시각화
plt.figure(figsize=(8, 5))
bars = plt.bar(x=recency_grouped.index, height=recency_grouped.values, color='#87CEEB')

# 그래프 막대 안에 백분율 값 표시
for bar, percentage in zip(bars, recency_percentage):
    plt.text(
        bar.get_x() + bar.get_width() / 2,  # 막대 중앙 x좌표
        bar.get_height() / 2,  # 막대 높이의 중간 y좌표
        f'{percentage:.1f}%',  # 백분율 값
        ha='center',  # 텍스트 정렬
        va='center',
        fontsize=15,
        color='white',
        fontweight='bold'  # 텍스트 굵게
    )

plt.xlabel('Recency', fontsize=14)
plt.ylabel('Number of Customers', fontsize=14)
plt.tight_layout()
plt.savefig('rfm_recency.png', dpi=300, bbox_inches='tight', transparent=True)

#### - Frequency

In [ ]:
# customer_unique_id 기준 order_id 수 count (Frequency : 고객 주문 횟수)
frequency_df = merged_cleaned.groupby('customer_unique_id').agg(
    frequency=('order_id', 'count'),  # 주문 횟수 계산
    product_ids=('product_id', lambda x: ', '.join(map(str, x.unique())))  # 고유한 product_id를 콤마로 구분해 나열
).reset_index()

# 주문 횟수가 없는 고객은 0으로 채우기
frequency_df['frequency'] = frequency_df['frequency'].fillna(0).astype(int)

# 주문 횟수 기준으로 정렬
frequency_df.sort_values(by='frequency', ascending=False, inplace=True)

# Frequency를 4개의 구간으로 나누기 (1, 2, 3, 4 이상)
bins = [0, 1, 2, 3, float('inf')]  # 구간 설정
labels = ['1', '2', '3', '4+']  # 레이블 설정
frequency_df['frequency_group'] = pd.cut(frequency_df['frequency'], bins=bins, labels=labels, right=True)

# 각 구간별 고객 수 및 비율 계산
frequency_grouped = frequency_df.groupby('frequency_group').size()
frequency_percentage = (frequency_grouped / frequency_grouped.sum()) * 100

# 파이 차트
plt.figure(figsize=(6, 6))
colors = plt.cm.Blues_r(np.linspace(0.3, 0.8, len(frequency_grouped)))
patches, texts = plt.pie(
    frequency_grouped, 
    labels=None,  # 레이블 제거
    autopct=None,  # 비율 표기 제거
    startangle=90, 
    colors=colors
)

# 범례에 표시할 영어 레이블 생성
legend_labels = [
    f'1 Time ({frequency_percentage[0]:.1f}%)',
    f'2 Times ({frequency_percentage[1]:.1f}%)',
    f'3 Times ({frequency_percentage[2]:.1f}%)',
    f'4+ Times ({frequency_percentage[3]:.1f}%)'
]

# 범례 추가 (오른쪽에 배치)
plt.legend(
    patches, 
    legend_labels, 
    loc='center left', 
    bbox_to_anchor=(1.02, 0.5),  # 범례 위치 조정
    ncol=1, 
    fontsize=14, 
    frameon=False
)

plt.tight_layout()
plt.subplots_adjust(right=0.7)  # 범례가 오른쪽에 있으므로 그래프 영역을 좁힘
plt.savefig('rfm_frequency.png', dpi=300, bbox_inches='tight', transparent=True)

#### - Monetary

In [ ]:
# 고객 고유 ID별로 총 지출(Monetary)을 계산
monetary_df = merged_cleaned.groupby('customer_unique_id').agg(
    monetary_usd=('price_usd', 'sum')
).reset_index()

# 지출 범위 구간 설정 (USD 기준)
bins_usd = [0, 12.02, 48.08, 96.16, 240.4, 3230.98]  # BRL에서 환산된 USD 범위
labels_usd = ['0-12', '13-48', '49-96', '97-240', '241-3231']  # 구간 레이블

# Monetary 값을 범위로 나누기
monetary_df['monetary_group'] = pd.cut(monetary_df['monetary_usd'], bins=bins_usd, labels=labels_usd, right=True)

# 각 지출 범위별 고객 수 계산
monetary_grouped = monetary_df.groupby('monetary_group').size()

# 시각화
plt.bar(x=monetary_grouped.index, height=monetary_grouped.values, color='#87CEEB')
plt.xlabel('Spending Range', fontsize=14)
plt.ylabel('Number of Customers', fontsize=14)
plt.tight_layout()
plt.savefig('rfm_monetary.png', dpi=300, bbox_inches='tight', transparent=True)

### RFM 분석

#### - RFM Scoring & Segmentation

In [ ]:
# Recency 점수 구간별 점수 설정 (1: 오래된 고객, 5: 최근 고객)
## 1. recency 값을 숫자로 매핑: 숫자 데이터로 변환해야 pd.cut() 사용 가능
recency_mapping = {
    '1month': 30,     # 1개월
    '2months': 60,    # 2개월
    '3months': 90,    # 3개월
    '1year': 365,     # 1년
    '1year+': 730     # 1년 이상
}
recency_df_f['recency_numeric'] = recency_df_f['recency'].map(recency_mapping)

## 2. pd.cut()으로 점수 구간 나누기
bins = [0, 30, 60, 90, 180, float('inf')]  # 구간 설정
labels = [5, 4, 3, 2, 1]  # 점수 (높을수록 최근 고객)

recency_df_f['recency_score'] = pd.cut(
    recency_df_f['recency_numeric'], bins=bins, labels=labels, right=True
)
recency_df_f['recency_score'] = recency_df_f['recency_score'].fillna(1)  # NaN 값 기본 점수로 대체

# Frequency 점수 구간별 점수 설정 (1: 적은 구매, 5: 많은 구매)
frequency_bins = [0, 1, 3, 5, 10, float('inf')]  # 구매 횟수 기준 구간
frequency_labels = [1, 2, 3, 4, 5]  # 점수 설정
frequency_df['frequency_score'] = pd.cut(
    frequency_df['frequency'], bins=frequency_bins, labels=frequency_labels, right=True
)

# Monetary 점수 구간별 점수 설정 (1: 적은 지출, 5: 높은 지출)
monetary_bins = [0, 12.02, 48.08, 96.16, 240.4, float('inf')]
monetary_labels = [1, 2, 3, 4, 5]
monetary_df['monetary_score'] = pd.cut(
    monetary_df['monetary_usd'], bins=monetary_bins, labels=monetary_labels, right=True
)
monetary_df['monetary_score'] = monetary_df['monetary_score'].fillna(1).astype(int)  # NaN 값 기본 점수로 대체

# RFM 데이터 통합 (Recency, Frequency, Monetary)
rfm_df = pd.merge(recency_df_f[['customer_unique_id', 'recency_score']], frequency_df[['customer_unique_id', 'frequency_score']], on='customer_unique_id')
rfm_df = pd.merge(rfm_df, monetary_df[['customer_unique_id', 'monetary_score']], on='customer_unique_id')

# 가중치 설정
recency_weight = 0.4
frequency_weight = 0.3
monetary_weight = 0.3

# RFM 점수 계산 (가중치 반영)
rfm_df['RFM_score'] = (
    rfm_df['recency_score'].astype(int) * recency_weight +
    rfm_df['frequency_score'].astype(int) * frequency_weight +
    rfm_df['monetary_score'].astype(int) * monetary_weight
)

# RFM 그룹 할당 (High Value, Medium Value, Low Value 등)
def assign_rfm_group(row):
    if row['RFM_score'] >= 4.5:
        return 'High Value'
    elif row['RFM_score'] >= 3:
        return 'Medium Value'
    else:
        return 'Low Value'
rfm_df['RFM_group'] = rfm_df.apply(assign_rfm_group, axis=1)

#### - RFM 점수 및 고객등급 시각화

In [ ]:
# RFM 점수 시각화

# y축 눈금 범위를 각 데이터 최대값을 기준으로 설정
y_max = max(rfm_df['recency_score'].value_counts().max(), 
            rfm_df['frequency_score'].value_counts().max(), 
            rfm_df['monetary_score'].value_counts().max())

# Recency 점수 분포
fig, ax = plt.subplots(figsize=(6, 5))
sns.histplot(rfm_df['recency_score'], kde=False, bins=5, ax=ax, color='#87CEEB', edgecolor='lightgray')
ax.set_xlabel('Recency Score', fontsize=14)
ax.set_ylabel('Count', fontsize=14)
ax.set_ylim(0, y_max * 1.2)
plt.tight_layout()
plt.savefig('rfm_score_recency.png', transparent=True)

# Frequency 점수 분포
fig, ax = plt.subplots(figsize=(6, 5))
sns.histplot(rfm_df['frequency_score'], kde=False, bins=5, ax=ax, color='#53abdb', edgecolor='lightgray')
ax.set_xlabel('Frequency Score', fontsize=14)
ax.set_ylabel('Count', fontsize=14)
ax.set_ylim(0, y_max * 1.2)
plt.tight_layout()
plt.savefig('rfm_score_frequency.png', transparent=True)

# Monetary 점수 분포
fig, ax = plt.subplots(figsize=(6, 5))
sns.histplot(rfm_df['monetary_score'], kde=False, bins=5, ax=ax, color='#1785c2', edgecolor='lightgray')
ax.set_xlabel('Monetary Score', fontsize=14)
ax.set_ylabel('Count', fontsize=14)
ax.set_ylim(0, y_max * 1.2)
plt.tight_layout()
plt.savefig('rfm_score_monetary.png', transparent=True)

In [ ]:
# RFM 고객등급 시각화

# RFM 그룹별 고객 수 계산
rfm_grouped = rfm_df.groupby('RFM_group').size()

# x축 레이블 순서 설정
rfm_group_order = ['Low Value', 'Medium Value', 'High Value']

# 총 고객 수와 비율 계산
total_customers = rfm_grouped.sum()
percentages = (rfm_grouped / total_customers) * 100

# 색상 설정 (막대별로 개별 지정)
bar_colors = ['#235699', '#4e9ad9', '#87CEEB']

plt.figure(figsize=(8, 6))
ax = sns.barplot(
    x=rfm_grouped.index,
    y=rfm_grouped.values,
    order=rfm_group_order
)
# 각 막대의 색상 개별 설정
for i, bar in enumerate(ax.patches):
    bar.set_color(bar_colors[i])

# 각 막대 위에 비율 표시
for p in ax.patches:
    height = p.get_height()
    percentage = (height / total_customers) * 100
    ax.text(
        p.get_x() + p.get_width() / 2,
        height + 10,  # 텍스트 위치 높이
        f'{percentage:.1f}%', 
        ha='center',
        va='bottom',
        fontsize=14
    )

plt.ylim(0, rfm_grouped.max() * 1.2)  # y축 상한을 최대값의 1.2배로 설정
plt.yticks(fontsize=12)
plt.xlabel('RFM Group', fontsize=14)
plt.ylabel('Number of Customers', fontsize=14)
plt.savefig('rfm_customer.png', transparent=True) 

## 주(state)별 총 매출 분석

In [ ]:
!pip install geopandas osmnx folium
import geopandas as gpd
import osmnx as ox
import folium
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
# 주별로 그룹화하여 매출 합계와 판매 건수 합계 계산
state_sales_summary = merged_cleaned.groupby('customer_state').agg({
    'price_usd': 'sum',    # 매출 합계
    'order_item_id': 'count'  # 판매 건수 합계
}).reset_index()

# 컬럼 이름 변경
state_sales_summary.rename(columns={
    'price_usd': 'total_sales_usd',
    'order_item_id': 'total_orders'
}, inplace=True)

# 브라질 주(State) 약자와 풀네임 매핑 딕셔너리
state_fullnames = {
    "AC": "Acre", "AL": "Alagoas", "AM": "Amazonas", "AP": "Amapa", "BA": "Bahia",
    "CE": "Ceara", "DF": "Federal District", "ES": "Espirito Santo", "GO": "Goias",
    "MA": "Maranhao", "MG": "Minas Gerais", "MS": "Mato Grosso do Sul", "MT": "Mato Grosso",
    "PA": "Para", "PB": "Paraiba", "PE": "Pernambuco", "PI": "Piaui", "PR": "Parana",
    "RJ": "Rio de Janeiro", "RN": "Rio Grande do Norte", "RO": "Rondonia", "RR": "Roraima",
    "RS": "Rio Grande do Sul", "SC": "Santa Catarina", "SE": "Sergipe", "SP": "Sao Paulo",
    "TO": "Tocantins"
}
# 매핑 딕셔너리를 사용해 새로운 컬럼 생성
state_sales_summary['state_fullname'] = state_sales_summary['customer_state'].map(state_fullnames)

# 결과를 CSV로 저장
state_sales_summary.to_csv('state_sales_summary.csv', index=False)

In [ ]:
# 브라질 주 경계 가져오기
brazil_states = ox.features_from_place("Brazil", tags={"admin_level": "4"})

# 필요한 열 선택 및 데이터 확인
brazil_states = brazil_states[['name', 'geometry']]

# 매출 데이터의 컬럼 이름 통일 및 수정
state_sales_summary.rename(columns={"state_fullname": "name"}, inplace=True)

# 데이터 병합 (풀네임 기준)
brazil_states = brazil_states.merge(state_sales_summary, on="name", how="left")

# 병합 후 빈 값 처리
brazil_states['total_sales_usd'] = brazil_states['total_sales_usd'].fillna(0)

In [ ]:
# 새로운 컬러맵 생성 
cmap = LinearSegmentedColormap.from_list(
    'custom_blues', 
    ['#dceefa', '#abd8f5', '#6db7e8', '#3576cc', '#05386b'],  # 컬러맵의 색상 리스트 (점점 진해지는 색상)
    N=256  # 색상 수 (256개 색상)
)

fig, ax = plt.subplots(1, 1, figsize=(12, 10))
brazil_states.plot(
    column='total_sales_usd',  # 매출 데이터
    cmap=cmap,  # 커스터마이즈된 컬러맵 적용
    legend=True,
    vmin=1000,  # 축 범위 최소값
    vmax=150000,  # 최대값
    ax=ax
)
ax.set_title("State-wise Sales in Brazil", fontsize=16)
ax.axis('off')

# 이미지 저장
plt.savefig("state_sales_brazil.png", dpi=300, bbox_inches='tight', transparent=True)

## 시간별/요일별 분석

### 제품 카테고리별 시간에 따른 구매량

In [ ]:
merged_cleaned['purchase_time'] = pd.to_datetime(merged_cleaned['order_purchase_timestamp'], errors='coerce').dt.hour
merged_cleaned['purchase_time'] = merged_cleaned['purchase_time'].fillna(0).astype(int)

purchase_by_hour = merged_cleaned.groupby('purchase_time')['price_usd'].sum()

plt.figure(figsize=(20, 3))
sns.lineplot(x=purchase_by_hour.index, y=purchase_by_hour.values, marker='o')

# plt.xlabel('Hour', fontsize=14)
# plt.ylabel('Product Category', fontsize=14)
plt.xticks(range(0, 24), fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('time_graph1.png', dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
result = merged_cleaned.groupby(['purchase_time', 'segmentation'])['price_usd'].sum().reset_index()
result.sort_values(by='price_usd', ascending=False)

# 시간(Hour) 단위로 그룹화하여 카테고리별 판매 횟수 계산
sales_count = merged_cleaned.groupby(['purchase_time', 'segmentation']).size().unstack(fill_value=0)

# purchase_time == 14에 해당하는 값만 추출하고, 내림차순으로 정렬하여 카테고리 순서를 결정
sorted_categories = sales_count.loc[14].sort_values(ascending=False).index

# 카테고리 순서만 재정렬
sales_count = sales_count[sorted_categories]

plt.figure(figsize=(20, 7))
sns.heatmap(sales_count.T, annot=True, cmap='Blues', fmt='d', annot_kws={"size": 10})
plt.xlabel('Hour', fontsize=14)
plt.ylabel('Product Category', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.show()

### 요일별 매출

In [ ]:
# dayofweek: 0,1,2,3,...을 dayofweek_eng : mon, tue, ...로 저장
# 요일 매핑을 딕셔너리로 정의
day_mapping = {
    0: 'Mon',
    1: 'Tue',
    2: 'Wed',
    3: 'Thu',
    4: 'Fri',
    5: 'Sat',
    6: 'Sun'
}

# dayofweek 값을 매핑하고 새로운 컬럼에 저장
merged_category['dayofweek_eng'] = merged_category['dayofweek'].map(day_mapping)

dayofweek_category = merged_category.groupby('dayofweek_eng')['price_usd'].sum().reset_index()

# 요일 순서 지정
day_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

# dayofweek_eng 열을 카테고리형 데이터로 변환
dayofweek_category['dayofweek_eng'] = pd.Categorical(
    dayofweek_category['dayofweek_eng'],
    categories=day_order,
    ordered=True
)

# 그래프 크기 설정
plt.figure(figsize=(10, 6))

# 바 그래프 생성
sns.barplot(
    data=dayofweek_category,
    x='dayofweek_eng',  # 요일
    y='price_usd',          # 가격
    color = 'skyblue'    # 색상 팔레트
)

# 그래프 제목 및 라벨 설정
plt.xlabel("", fontsize=12)
plt.ylabel("Total Sales", fontsize=12)

# 레이아웃 조정 및 출력
plt.tight_layout()
# plt.savefig('/content/drive/MyDrive/project/그래프/23_dayofweek_sales.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

## 결제 방식 분석

In [ ]:
# 'order_id'를 기준으로 'merged_cleaned'와 'payment' 데이터프레임 병합
merged_payment = pd.merge(merged_cleaned, payment, on='order_id', how='inner')

# 필요한 컬럼만 선택
merged_payment = merged_payment[['order_id', 'payment_type', 'payment_installments']]

# 'payment_type'의 값 개수와 비율을 내림차순으로 계산
payment_type_counts = merged_payment['payment_type'].value_counts(ascending=False)
payment_type_percentage = merged_payment['payment_type'].value_counts(ascending=False, normalize=True) * 100

# 두 값을 합쳐서 DataFrame 형태로 출력
payment_type_summary = pd.DataFrame({
    'count': payment_type_counts,
    'percentage': payment_type_percentage
})

# 시각화 
fig, ax1 = plt.subplots(figsize=(10, 6))
sns.barplot(x=payment_type_summary.index, y=payment_type_summary['count'], ax=ax1, color='#87CEEB', alpha=0.6)
ax1.set_xlabel('Payment Type', fontsize=16)
ax1.set_ylabel('Count', fontsize=14)
ax1.tick_params(axis='y')

ax2 = ax1.twinx()  # 두 번째 y축 생성

# 'percentage'를 꺾은선그래프로 시각화
sns.lineplot(x=payment_type_summary.index, y=payment_type_summary['percentage'], ax=ax2, color='#8B0000', marker='o')
ax2.set_ylabel('Percentage (%)', fontsize=14)
ax2.tick_params(axis='y')

plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('payment_type.png', transparent=True)